In [ ]:
import re
import dynamo as dyn
import numpy as np 
import pandas as pd
from scipy.spatial.distance import euclidean 
import matplotlib.pyplot as plt
%matplotlib inline
import inspect

In [ ]:
adata=dyn.read_h5ad('../data/a549_tgfb1.h5ad')
meta_path = '../data/a549_tgfb1_meta.csv'
meta = pd.read_csv(meta_path)
adata

In [ ]:
cc_gene_list=np.loadtxt('../gene_lists/cc_gene_list.txt',dtype=str)
emt_gene_list=np.loadtxt('../gene_lists/emt_genes_weikang.txt',dtype=str)

In [ ]:
CellIDs=np.array(meta["Unnamed: 0"])+'x'
for ID in range(len(CellIDs)):
    #This is needed to make the cell ids have the same syntax as the loom files 
    CellIDs[ID]=re.sub('x',"x-",CellIDs[ID],count=1)
    CellIDs[ID]=re.sub('_',":",CellIDs[ID])

meta['Unnamed: 0']=CellIDs

cells=meta['Unnamed: 0'].to_numpy()

treatment=np.array([[meta['Time'][np.squeeze(np.argwhere(cells==cell))]][0] for cell in adata.obs_names])

adata.obsm['treatment']=treatment
adata

In [ ]:
stable=['0d']

stable=np.isin(adata.obsm['treatment'],stable)
stable_idx=np.squeeze(np.argwhere(stable==True))

adata=adata[stable_idx,:]
adata

In [ ]:
dyn.pp.recipe_monocle(adata, n_top_genes=2000,keep_filtered_genes=False)
dyn.tl.dynamics(adata)
dyn.tl.reduceDimension(adata,basis='pca')
dyn.tl.cell_velocities(adata,basis='pca')

In [ ]:
def plot_V(X, V, dim1=0, dim2=1, create_figure=False, figsize=(6, 6), **kwargs):
    if create_figure:
        plt.figure(figsize=figsize)
    plt.quiver(X[:, dim1], X[:, dim2], V[:, dim1], V[:, dim2])
    
def plot_X(X, dim1=0, dim2=1, create_figure=False, figsize=(6, 6), **kwargs):
    if create_figure:
        plt.figure(figsize=figsize)
    plt.scatter(X[:, dim1], X[:, dim2], **kwargs)

In [ ]:
vf = dyn.tl.VectorField(adata,basis='pca', dims= 50,return_vf_object=True, pot_curl_div=False)
Q = adata.uns['PCs'][:, :50]
vf_raw_lambda =  lambda x: vf.func(x) @ Q.T

In [ ]:
gene1_idx=np.argwhere(adata.var_names=='VIM').squeeze()
gene2_idx=np.argwhere(adata.var_names=='FN1').squeeze()

vf_raw=vf_raw_lambda(adata.obsm['X_pca'])

two_gene_velocities=np.vstack((vf_raw[:,gene1_idx],vf_raw[:,gene2_idx])).T

In [ ]:
X_emb=adata.obsm['X_pca']
Uc=two_gene_velocities

U_grid, X_grid = dyn.tl.smoothen_drift_on_grid(X_emb[:, :2], Uc[:, :2], 30, k=50, smoothness=0.5)

In [ ]:
plot_X(X_emb, create_figure=True, figsize=(12, 6))
plot_V(X_grid, U_grid, facecolor='k')

In [ ]:
centroid=np.mean(adata.obsm['X_pca'][:,:2],axis=0)
delta_x=np.array([euclidean(centroid,cell) for cell in adata.obsm['X_pca'][:,:2]])

closest_sample_to_centroid=np.argmin(delta_x)

cluster_members=delta_x<4.5

print(sum(cluster_members))

plt.scatter(adata.obsm['X_pca'][:,0],adata.obsm['X_pca'][:,1],c=cluster_members)
plt.scatter(centroid[0],centroid[1])

In [ ]:
temp_adata=dyn.read_h5ad('../data/a549_tgfb1.h5ad')
meta_path = '../data/a549_tgfb1_meta.csv'
meta = pd.read_csv(meta_path)
temp_adata

In [ ]:
CellIDs=np.array(meta["Unnamed: 0"])+'x'
for ID in range(len(CellIDs)):
    #This is needed to make the cell ids have the same syntax as the loom files 
    CellIDs[ID]=re.sub('x',"x-",CellIDs[ID],count=1)
    CellIDs[ID]=re.sub('_',":",CellIDs[ID])

meta['Unnamed: 0']=CellIDs

cells=meta['Unnamed: 0'].to_numpy()

treatment=np.array([[meta['Time'][np.squeeze(np.argwhere(cells==cell))]][0] for cell in temp_adata.obs_names])

temp_adata.obsm['treatment']=treatment
temp_adata

In [ ]:
stable=['0d']

stable=np.isin(temp_adata.obsm['treatment'],stable)
stable_idx=np.squeeze(np.argwhere(stable==True))

temp_adata=temp_adata[stable_idx,:]
temp_adata

In [ ]:
cluster_adata=temp_adata[cluster_members]
cluster_adata

In [ ]:
dyn.pp.recipe_monocle(cluster_adata, n_top_genes=2000,keep_filtered_genes=False)
dyn.tl.dynamics(cluster_adata)
dyn.tl.reduceDimension(cluster_adata,basis='pca')
dyn.tl.cell_velocities(cluster_adata,basis='pca')

In [ ]:
vf = dyn.tl.VectorField(cluster_adata,basis='pca', dims= 50,return_vf_object=True, pot_curl_div=False)
Q = cluster_adata.uns['PCs'][:, :50]
vf_raw_lambda =  lambda x: vf.func(x) @ Q.T

In [ ]:
gene1_idx=np.argwhere(cluster_adata.var_names=='VIM').squeeze()
gene2_idx=np.argwhere(cluster_adata.var_names=='FN1').squeeze()

vf_raw=vf_raw_lambda(cluster_adata.obsm['X_pca'])

cluster_two_gene_velocities=np.vstack((vf_raw[:,gene1_idx],vf_raw[:,gene2_idx])).T

In [ ]:
X_emb=cluster_adata.obsm['X_pca']
Uc=cluster_two_gene_velocities

U_grid, X_grid = dyn.tl.smoothen_drift_on_grid(X_emb[:, :2], Uc[:, :2], 30, k=50, smoothness=0.5)

In [ ]:
plot_X(X_emb, create_figure=True, figsize=(12, 6))
plot_V(X_grid, U_grid, facecolor='k')
plt.scatter(adata.obsm['X_pca'][closest_sample_to_centroid,0],
            adata.obsm['X_pca'][closest_sample_to_centroid,1],
            label='sample closest to artifical centroid')

plt.title('Velocity Arrows from VIM and FN1')
plt.legend()